This is a script for probing the capabilities of the rectv algorithm by Nikitin. This script should be run with py39 conda environment. First lets load some packages we need. 

In [3]:
import rectv_gpu
import numpy as np
import dxchange
import matplotlib.pyplot as plt
import ipywidgets as widgets
from os import listdir
from os.path import join
from ImageSliceViewer3D import *
import tomopy

I have used the script in PhantomGenerato "xraysim.py" to simulate some dynamic data. The data generation has a bit of a finicky problem requiring some transposing for the data to work. Good thing is that the blender file can be changed to create more/less motion. We can run the simulation like this: blender input/particle_test.blend --frame-start 1 --frame-end 180 -o output/Simulation/ --background --python xraysim.py -- --numberofangles 180 --anglesperframe 1 -n Empty


In [4]:
filepath = "PhantomGenerator/output/RECTVData"
files = [file for file in np.sort(listdir(filepath)) if file.endswith(".npy")]
height, width = np.load(join(filepath,files[0])).shape
depth = len(files)
volume = np.zeros((depth, height, width))
for i,file in enumerate(files):
    volume[i,:,:] = np.load(join(filepath,file))

volume = volume.astype('float32')
#Need a bit of trasposing for the data to work....
#data = np.swapaxes(volume, 1,2)
data = np.swapaxes(volume, 1,0)
[nz, ntheta, n] = data.shape
nz, ntheta, n

(128, 2400, 128)

ImageSliceViewer can be used to view the data. You can see it once you up the slice number

In [5]:
ImageSliceViewer3D(data)

In [6]:
rot_center = n//2

In [7]:
#print(theta[100:200])
theta = np.linspace(0, 8*np.pi, ntheta, endpoint=False).astype('float32')

By using tomopy for reconstruction we see that static parts are reconstructed but moving particle is causing artifacts.

In [8]:
recon = tomopy.recon(np.swapaxes(data, 0,1), theta, center=rot_center, algorithm='fbp', ncore=32)
ImageSliceViewer3D(recon)

We could try and load rectv test data just to see how this works with tomopy gridrec - it will also give us an understandig of how the data is indexed

In [9]:
def takephi(ntheta):
    m = 32 # number of basis functions
    [x, y] = np.meshgrid(np.arange(-ntheta//2, ntheta//2), np.arange(-m//2, m//2))
    phi = np.zeros([m, 2*ntheta], dtype='float32')
    phi[:, ::2] = np.cos(2*np.pi*x*y/ntheta)/np.sqrt(ntheta)
    phi[:, 1::2] = np.sin(2*np.pi*x*y/ntheta)/np.sqrt(ntheta)
    return phi

In [11]:
#lambda0 = 1e-3  # regularization parameter 1
#lambda1 = 1  #regularization parameter 2

In [12]:
niter = 32  # number of ADMM iterations
titer = 4  # number of inner tomography iterations

In [13]:
nzp = 4 # number of slices to process simultaniously by gpu
ngpus = 1 # number of gpus

In [14]:
phi = takephi(ntheta) 
m = phi.shape[0] # number of basis functions
phi, m

(array([[ 2.04124153e-02, -3.99967965e-17,  2.03945097e-02, ...,
          1.70806679e-03,  2.03945097e-02,  8.54783226e-04],
        [-2.04124153e-02,  1.10016464e-16, -2.03966778e-02, ...,
         -1.60153955e-03, -2.03966778e-02, -8.01387650e-04],
        [ 2.04124153e-02, -3.49971978e-17,  2.03987062e-02, ...,
          1.49496843e-03,  2.03987062e-02,  7.47986545e-04],
        ...,
        [-2.04124153e-02,  4.00220652e-17, -2.04005931e-02, ...,
          1.38835632e-03, -2.04005931e-02,  6.94580376e-04],
        [ 2.04124153e-02,  3.49971978e-17,  2.03987062e-02, ...,
         -1.49496843e-03,  2.03987062e-02, -7.47986545e-04],
        [-2.04124153e-02, -1.10016464e-16, -2.03966778e-02, ...,
          1.60153955e-03, -2.03966778e-02,  8.01387650e-04]], dtype=float32),
 32)

In [ ]:
cl = rectv_gpu.Solver(n, ntheta, m, nz, nzp, ngpus)

In [ ]:
#try varying the last arguments
rtv = cl.recon(data, theta, phi, rot_center=rot_center, niter=200, titer=4,lambda0=1e-2, lambda1=100)

In [ ]:
ImageSliceViewer3D(rtv[0])

In [ ]:
ImageSliceViewer3D(rtv[:,0])

In [ ]:
rtv.shape


In [ ]:
nz, ntheta, n


In [ ]:

slice = 58
time = 2
plt.imshow(rtv[slice,time])
plt.show()

In [ ]:
phi.shape

In [ ]:
48000/16


In [ ]:
ntheta=2400
m = 16
[x, y] = np.meshgrid(np.arange(-ntheta//2, ntheta//2), np.arange(-m//2, m//2))

In [ ]:
x, y

In [ ]:
phi = np.zeros([m, 2*ntheta], dtype='float32')

In [15]:
phi.shape

(32, 4800)